In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re

import matplotlib
import matplotlib.cm as mplcm
import matplotlib.colors as colors

In [7]:
state_to_stfips_path ="/project/biocomplexity/COVID-19_commons/2019-nCoV_stat-analyses/misc_data/stfips_to_state.csv"
state_to_stfips_df = pd.read_csv(state_to_stfips_path, header=None, dtype={"2":str})
state_to_stfips_df[2] = state_to_stfips_df[2].astype(str).str.zfill(2)
state_to_stfips = state_to_stfips_df.set_index(0)[2].to_dict()
abbr_to_stfips = state_to_stfips_df.set_index(1)[2].to_dict()

# join_data = pd.read_csv("../variant_data/epiToPublic_USA.tsv", sep="\t", header=None)
join_data = pd.read_table("../variant_data/epiToPublic.tsv", header=None, low_memory=True)
join_data.columns = ["EPI", "Accession", "name", "date"]

accession_to_epi = join_data.set_index("Accession")["EPI"].to_dict()
gis_accessions = join_data.Accession.tolist()

def convert_state_stfips(state):
    try:
        return state_to_stfips[state]
    except:
        return np.nan
    
def convert_abbr_stfips(abbr):
    try:
        return abbr_to_stfips[abbr]
    except:
        return np.nan
        
def convert_accession_epi(accession):
    try:
        return accession_to_epi[accession]
    except:
        return np.nan

metadata = pd.read_csv("../variant_data/metadata.tsv", sep="\t", parse_dates = ["Collection date", "Submission date"])

def get_nth_slash(row, n):
    try:
        return row.split("/")[n].strip()
    except:
        return np.nan
    
for i in range(4):
    metadata[f"Location_{i}"] = metadata["Location"].apply(lambda row: get_nth_slash(row, i))
metadata = metadata.rename(columns={"Location_1":"country", "Location_2":"state"})
metadata = metadata[metadata["country"] == "USA"]
metadata["alias"] = metadata["Virus name"].apply(lambda row: "/".join(row.split("/")[2:]))
# metdata = metadata.rename(columns={"Accession ID":"EPI"})

ncbi = pd.read_csv("../variant_data/sequences.csv", parse_dates=["Release_Date"])
ncbi["is_USA"] = ncbi["Geo_Location"].apply(lambda row: True if "USA" in str(row) else False)
ncbi = ncbi[ncbi.is_USA]
all_usa_ncbi = ncbi.shape
print(ncbi.shape[0], "NCBI from the US")

ncbi = ncbi.dropna(subset=["GenBank_Title"])
ncbi["alias"] = ncbi["GenBank_Title"].apply(lambda row: row.split(",")[0].split(" isolate ")[-1].split("USA/")[-1])

accession_to_epi = join_data.set_index("Accession")["EPI"].to_dict()
ncbi["EPI"] = ncbi.Accession.apply(convert_accession_epi)

ncbi = ncbi[(~ncbi.EPI.isin(metadata["Accession ID"])) & (~ncbi.alias.isin(metadata.alias))]
only_ncbi = ncbi.shape
ncbi = ncbi[~ncbi.USA.isna()][["Collection_Date", "Pangolin", "USA"]]
ncbi.columns = ["date", "variant", "state"]
ncbi.date = pd.to_datetime(ncbi.date)
ncbi.state = ncbi.state.apply(convert_abbr_stfips)
ncbi = ncbi.dropna()
print(all_usa_ncbi[0] - only_ncbi[0], "in NCBI and GISAID")
print(only_ncbi[0], "in NCBI only")

metadata = metadata[["Collection date", "Pango lineage", "state"]]
metadata.columns = ["date", "variant", "state"]
metadata.date = pd.to_datetime(metadata.date)
metadata.state = metadata.state.apply(convert_state_stfips)
metadata = metadata.dropna()

151312 NCBI from the US
142593 in NCBI and GISAID
8719 in NCBI only


In [8]:
combine_lineages = False

variants = pd.concat([metadata, ncbi], ignore_index=True)
# variants = variants[variants.date >= "2020-02-27"] #First day with >10 samples
vocs = ['B.1.1.7',
        'B.1.351',
        'P.1',
        'B.1.617.1',
        'B.1.617.2',
        'B.1.427',
        'B.1.429',
        'B.1.526',
        'B.1.526.1',
        'B.1.526.2']
variants.variant = variants.variant.apply(lambda row: row if row in vocs else "other")

if combine_lineages:
    variants.variant = variants.variant.apply(lambda row: re.sub(".[12]$", "", row))
variants.variant.unique()

variant_names = variants.variant.unique()
NUM_COLORS = len(variant_names)

cm = plt.get_cmap('terrain')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
terrain_variant_colors = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

cm = plt.get_cmap('tab20b')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
tab_variant_colors = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

In [12]:
va_variants = variants[variants.state == "51"]
